In [12]:
# download libraries
import os
!apt-get install openjdk-21-jre-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-21-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-21-openjdk-amd64/bin/java
!java -version
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
#from google.colab import drive

zsh:1: command not found: apt-get


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


zsh:1: command not found: update-alternatives


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


openjdk version "23.0.2" 2025-01-21
OpenJDK Runtime Environment Homebrew (build 23.0.2)
OpenJDK 64-Bit Server VM Homebrew (build 23.0.2, mixed mode, sharing)


In [13]:
# read in wordcount pkl files
#drive.mount('/content/drive', force_remount=True)
import os
#pkl_folder = '/content/drive/My Drive/pkl files'
pkl_files = [os.path.join('.', f) for f in os.listdir('.') if f.endswith('.pkl')]

for pkl in pkl_files:
  print(pkl)

./social_media_word_counts.pkl
./ethical_ai_query_word_counts.pkl
./metabolic_fasting_word_counts.pkl
./healthy_gut_word_counts.pkl
./battery_word_counts.pkl
./exercise_word_counts.pkl
./climate_change_word_counts.pkl
./urbanization_wildfire_word_counts.pkl
./genetic_engineering_word_counts.pkl
./smoking_query_word_counts.pkl


In [14]:
import pickle
path = '.'
topics = {'Effects of smoking on lung cancer': 'smoking_query_word_counts.pkl',
       'Ethical implications of AI': 'ethical_ai_query_word_counts.pkl',
       'Climate change and wildfires': 'climate_change_word_counts.pkl',
       'Effects of Fasting on Metabolic Health': 'metabolic_fasting_word_counts.pkl',
       'Psychological effects of social media on children and teens': 'social_media_word_counts.pkl',
       'Impacts of urbanization on wildlife biodiversity': 'urbanization_wildfire_word_counts.pkl',
       'Battery technology advancements for electric vehicles': 'battery_word_counts.pkl',
       'Genetic engineering as treatment for genetic disorders': 'genetic_engineering_word_counts.pkl',
       'Benefits of a healthy gut': 'healthy_gut_word_counts.pkl',
       'How exercise affects mental health': 'exercise_word_counts.pkl'
       }

all_data = {}
# load query dict
for topic, filename in topics.items():
        full_path = os.path.join(path, filename)
        with open(full_path, "rb") as f:
          all_data[topic] = pickle.load(f)


In [15]:
len(all_data)

10

In [16]:
# get doc for a given topic and then get 30 more irrelevant from diff topic to make and set that relevant to 0
def get_query_corpus(all_data, query):
  # doc for given query
  query_corpus = all_data[query]

  # get other docs for a query
  other_docs = {}
  for q, docs in all_data.items():
    if q != query:
      other_docs.update(docs)

  # randomly sample 30 keys from other_docs
  sampled_other_docs = np.random.choice(list(other_docs.keys()), 30, replace=False)

  # get sampled docs and update query_corpus with irrelevant and repalce values
  for key in sampled_other_docs:
    irrelevant_doc = other_docs[key]
    irrelevant_doc['relevance'] = 0 # update relevance to 0
    query_corpus[key] = (irrelevant_doc) # update in query_corpus

  return query_corpus

In [17]:
# create baseline rerank function
# import sentence transformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# import sentence transformer
sentence_transformer = SentenceTransformer('all-MiniLM-L6-v2')

def reranking_model(query):
  # this reranking model ONLY encodes using sentence transformer to calculate cosine similarity between query and docs + feel free to update with using other keys like citations/references/etc.
  query_corpus = get_query_corpus(all_data, query)
  doc_ids = list(query_corpus.keys())
  #print(query_corpus)


  # get document tokens as list and convery to str to pass into sentence transformer encoder (cant pass in counter dict)
  document_tokens = [doc['tokens'] for doc in query_corpus.values()]

  def counter_to_str(counter):
    return ' '.join([token for token, freq in counter.items() for _ in range(freq)])

  document_str = [counter_to_str(counter) for counter in document_tokens]

  # encode
  query_embedding = sentence_transformer.encode(query)
  document_embeddings = sentence_transformer.encode(document_str)


  # calculate cosine similarity
  cosine_similarities = cosine_similarity([query_embedding], document_embeddings)[0]

  # incorporate citation and reference weights
  adjusted_scores = []
  citation_weight = 0.005
  reference_weight = 0.01
  for idx, doc_id in enumerate(doc_ids):
      doc_info = query_corpus[doc_id]
      citation_count = doc_info.get('citations', 0)
      reference_count = doc_info.get('references', 0)
      boost = citation_weight * citation_count + reference_weight * reference_count
      adjusted_score = cosine_similarities[idx] + boost
      adjusted_scores.append((str(doc_id), float(adjusted_score)))

  # sort reranked list by adjusted score
  adjusted_scores.sort(key=lambda x: x[1], reverse=True)
  return adjusted_scores
  # get reranked list and rank in decending
  #reranked = list(zip([str(doc_id) for doc_id in doc_ids], [float(score) for score in cosine_similarities]))
  #reranked.sort(key=lambda x: x[1], reverse=True)
  #return reranked

# Rerank for a specific query
reranking_model('Effects of smoking on lung cancer')
#lung_cancer_reranked = reranking_model('Effects of smoking on lung cancer')
#print(lung_cancer_reranked)

[('https://www.semanticscholar.org/paper/76ded5364597a6b4abadd6a155f67b262c2c3f7c',
  42.4698486328125),
 ('https://www.semanticscholar.org/paper/781ce8ad18e51bde11d0ee1446816ea55440e611',
  4.368592262268066),
 ('https://www.semanticscholar.org/paper/7425f3ca048b011e0680b1ad9c3df0d034a49ccc',
  3.9876720905303955),
 ('https://www.semanticscholar.org/paper/b3aedeee136cb217c9c27d1102db1d23cc70742e',
  2.9002346992492676),
 ('https://www.semanticscholar.org/paper/4e5df8511676efd686361203b9e92ad907586050',
  2.759922742843628),
 ('https://www.semanticscholar.org/paper/1376df0751944d34c7bf8b04c2eb16b7fc221a6d',
  2.7265398502349854),
 ('https://www.semanticscholar.org/paper/096e515a15a6ad294557403486446d6b551052f4',
  2.725175380706787),
 ('https://www.semanticscholar.org/paper/dd81d8d9292d5da251299cc8c38c620a18125d1e',
  2.6210689544677734),
 ('https://www.semanticscholar.org/paper/cdba593e242cfe5e00a8c682e761e80d7ebf0ede',
  2.5750246047973633),
 ('https://www.semanticscholar.org/paper/e

# Metrics

In [18]:
def calculate_evaluation_metrics(query, reranked_results, query_corpus, k_map=50, k_dcg=20):
    # Documents with relevance > 0 are considered relevant
    relevant_docs = {doc_id for doc_id, doc in query_corpus.items()
                     if 'relevance' in doc and doc['relevance'] > 0}

    # Get relevance scores for reranked results
    relevance_scores = []
    for doc_id, _ in reranked_results:
        if doc_id in query_corpus and 'relevance' in query_corpus[doc_id]:
            relevance_scores.append(query_corpus[doc_id]['relevance'])
        else:
            relevance_scores.append(0)

    # Calculate Average Precision@k_map
    ap_k = average_precision(relevant_docs, reranked_results, k=k_map)

    # Calculate DCG@k_dcg
    dcg_k = dcg(relevance_scores, k=k_dcg)

    return {
        'query': query,
        f'AP@{k_map}': ap_k,
        f'DCG@{k_dcg}': dcg_k
    }

def average_precision(relevant_docs, predicted_docs, k=50):
    if not relevant_docs:
        return 0.0

    predicted_docs = predicted_docs[:k]
    precision_sum = 0.0
    relevant_found = 0

    for i, (doc_id, _) in enumerate(predicted_docs):
        if doc_id in relevant_docs:
            relevant_found += 1
            precision_sum += relevant_found / (i + 1)

    return precision_sum / len(relevant_docs) if relevant_docs else 0.0


def dcg(relevance_scores, k=20):
    relevance_scores = relevance_scores[:k]
    dcg = 0.0
    for i, score in enumerate(relevance_scores):
        dcg += score / np.log2(i + 2)
    return dcg

def evaluate_all_queries(queries, all_data, reranking_model, k_map=50, k_dcg=20):
    results = []
    ap_scores = []
    dcg_scores = []

    for query in queries:
        query_corpus = get_query_corpus(all_data, query)
        reranked_results = reranking_model(query)
        metrics = calculate_evaluation_metrics(query, reranked_results, query_corpus, k_map, k_dcg)

        ap_scores.append(metrics[f'AP@{k_map}'])
        dcg_scores.append(metrics[f'DCG@{k_dcg}'])
        results.append(metrics)

    mean_ap = np.mean(ap_scores) if ap_scores else 0.0
    mean_dcg = np.mean(dcg_scores) if dcg_scores else 0.0

    return results, mean_ap, mean_dcg

queries = ['Effects of smoking on lung cancer',
       'Ethical implications of AI',
       'Climate change and wildfires',
       'Effects of Fasting on Metabolic Health',
       'Psychological effects of social media on children and teens',
       'Impacts of urbanization on wildlife biodiversity',
       'Battery technology advancements for electric vehicles',
       'Genetic engineering as treatment for genetic disorders',
       'Benefits of a healthy gut',
       'How exercise affects mental health']
results, map_score, mean_dcg_score = evaluate_all_queries(queries, all_data, reranking_model)

for res in results:
  print("Query:", res['query'])
  print("Average Precision @ 50:", res['AP@50'])
  print("Discounted Cumulative Gain @ 20:", res["DCG@20"])

print("MAP@50:", map_score)
print("Mean DCG@20:", mean_dcg_score)




Query: Effects of smoking on lung cancer
Average Precision @ 50: 0.1509620317044079
Discounted Cumulative Gain @ 20: 3.9780163286912025
Query: Ethical implications of AI
Average Precision @ 50: 0.17755512846891602
Discounted Cumulative Gain @ 20: 3.5017741718336963
Query: Climate change and wildfires
Average Precision @ 50: 0.3580185833719014
Discounted Cumulative Gain @ 20: 6.160536200080017
Query: Effects of Fasting on Metabolic Health
Average Precision @ 50: 0.15216624206756799
Discounted Cumulative Gain @ 20: 3.624598993897811
Query: Psychological effects of social media on children and teens
Average Precision @ 50: 0.20808928661544174
Discounted Cumulative Gain @ 20: 3.382444954802684
Query: Impacts of urbanization on wildlife biodiversity
Average Precision @ 50: 0.15098810635806847
Discounted Cumulative Gain @ 20: 1.7554487843234847
Query: Battery technology advancements for electric vehicles
Average Precision @ 50: 0.033492409873988827
Discounted Cumulative Gain @ 20: 1.30253643

In [19]:
import numpy as np
from sklearn.metrics import average_precision_score, precision_recall_curve, auc
from scipy.stats import gmean

def dcg(rels):
    return sum(rel / np.log2(idx + 2) for idx, rel in enumerate(rels))

def ndcg_at_k(true, pred, k):
    true = np.array(true)
    ideal = sorted(true, reverse=True)[:k]
    pred_rels = true[pred[:k]]
    return dcg(pred_rels) / (dcg(ideal) + 1e-8)

def recall_at_k(true, pred, k):
    true = np.array(true) > 0
    pred = np.array(pred)[:k]
    return np.sum(true[pred]) / np.sum(true) if np.sum(true) > 0 else 0.0

def auc_pr(true_labels, predicted_scores):
    precision, recall, _ = precision_recall_curve(true_labels, predicted_scores)
    return auc(recall, precision)

def bpref(relevance_scores):
    relevant_indices = [i for i, rel in enumerate(relevance_scores) if rel > 0]
    non_relevant_above = 0
    for r in relevant_indices:
        non_relevant_above += sum(1 for i in range(r) if relevance_scores[i] == 0)
    return 1 - (non_relevant_above / (len(relevant_indices) * len(relevance_scores))) if relevant_indices else 0.0

def r_precision(relevance_scores):
    R = sum(relevance_scores)
    return sum(relevance_scores[:R]) / R if R > 0 else 0.0

def mean_reciprocal_rank(true, pred):
    for idx, doc_idx in enumerate(pred, 1):
        if true[doc_idx] > 0:
            return 1.0 / idx
    return 0.0

def calculate_extended_metrics(query, reranked_results, query_corpus, k_ndcg=30, k_recall=60):
    # Build relevance vector
    relevance_list = []
    doc_id_to_idx = {}
    for i, (doc_id, _) in enumerate(reranked_results):
        rel = query_corpus.get(doc_id, {}).get('relevance', 0)
        relevance_list.append(rel)
        doc_id_to_idx[doc_id] = i

    binary_rels = [1 if r > 0 else 0 for r in relevance_list]
    scores = [score for _, score in reranked_results]
    ranked_indices = np.arange(len(reranked_results))

    return {
        'query': query,
        f'Recall@{k_recall}': recall_at_k(binary_rels, ranked_indices, k_recall),
        f'NDCG@{k_ndcg}': ndcg_at_k(binary_rels, ranked_indices, k_ndcg),
        'AUC-PR': auc_pr(binary_rels, scores),
        'bpref': bpref(relevance_list),
        'R-Precision': r_precision(relevance_list),
        'MRR': mean_reciprocal_rank(binary_rels, ranked_indices),
        'AP': average_precision_score(binary_rels, scores)
    }

def evaluate_all_queries_extended(queries, all_data, reranking_model, k_ndcg=30, k_recall=60):
    results = []
    ap_scores = []

    for query in queries:
        query_corpus = get_query_corpus(all_data, query)
        reranked_results = reranking_model(query)
        if not reranked_results:
            continue
        metrics = calculate_extended_metrics(query, reranked_results, query_corpus, k_ndcg, k_recall)
        ap_scores.append(metrics['AP'])
        results.append(metrics)

    gmap_score = gmean(ap_scores) if ap_scores else 0.0
    return results, gmap_score

# Example usage
results, gmap_score = evaluate_all_queries_extended(queries, all_data, reranking_model)

for res in results:
    print("\nQuery:", res['query'])
    print("Recall@60:        ", res['Recall@60'])
    print("NDCG@30:          ", res['NDCG@30'])
    print("AUC-PR:           ", res['AUC-PR'])
    print("bpref:            ", res['bpref'])
    print("R-Precision:      ", res['R-Precision'])
    print("MRR:              ", res['MRR'])
    print("AP:               ", res['AP'])

print("\nGMAP: ",gmap_score)



Query: Effects of smoking on lung cancer
Recall@60:         0.4375
NDCG@30:           0.11404757491222692
AUC-PR:            0.1055029248499261
bpref:             0.6545877659574468
R-Precision:       0.16666666666666666
MRR:               0.07142857142857142
AP:                0.11564968838826398

Query: Ethical implications of AI
Recall@60:         0.3
NDCG@30:           0.13711668721949832
AUC-PR:            0.11412817769672193
bpref:             0.5293413173652695
R-Precision:       0.18181818181818182
MRR:               0.3333333333333333
AP:                0.1296547289632921

Query: Climate change and wildfires
Recall@60:         0.5384615384615384
NDCG@30:           0.27046500698389203
AUC-PR:            0.1670298929843484
bpref:             0.6102251407129455
R-Precision:       0.21052631578947367
MRR:               1.0
AP:                0.1757302680222087

Query: Effects of Fasting on Metabolic Health
Recall@60:         0.25
NDCG@30:           0.15099568635473729
AUC-PR:    